# 01A Data Cleansing Weather

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
%matplotlib inline

## Read Data

In [2]:
weather_df = pd.read_csv('../west_nile/west_nile/input/weather.csv')

## One Hot Encode CodeSum Data

In [3]:
codes = {code: [0]*weather_df.shape[0] for codes in weather_df.CodeSum.unique() for code in codes.split()}

weather_df = weather_df.join(pd.DataFrame(codes))
for i, codesum in enumerate(weather_df.CodeSum):
    for code in codesum.split():
        weather_df.at[i, code] = 1
        
codes_df = weather_df.groupby('Date', as_index=True)[list(codes.keys())].max()

## Export codes_df

In [4]:
with open('../Cleansed_Data/codes_df.pkl', 'wb') as f:
    pickle.dump(codes_df, f)

----------

# New Weather Processing

## Import weather data

In [5]:
weather = pd.read_csv('../west_nile/west_nile/input/weather.csv')

## Impute missing Tavg Values and Convert column to Int

In [6]:
missing_Tavg = weather[weather.Tavg == 'M'].index

weather.loc[missing_Tavg, 'Tavg'] = round((weather.loc[missing_Tavg, 'Tmax'] 
                                              + weather.loc[missing_Tavg, 'Tmin'])/2)
weather.Tavg = weather.Tavg.astype(int)

## Impute missing AvgSpeed and Convert column to float

In [7]:
median_wind = weather[weather.AvgSpeed!='M'].AvgSpeed.median()
weather.AvgSpeed.replace('M', median_wind, inplace=True)
weather.AvgSpeed = weather.AvgSpeed.astype(float)

## Create Temperature Columns in Celsius

In [8]:
def celsius(x):
    c = ((x - 32) * 5.0)/9.0
    return float(c)

In [9]:
weather['TavgC'] = weather['Tavg'].apply(celsius)
weather['TminC'] = weather['Tmin'].apply(celsius)
weather['TmaxC'] = weather['Tmax'].apply(celsius)

## Calculate Relative Humidity

In [10]:
weather['relative_humidity'] = 100 * (np.exp((17.625 * weather.DewPoint.apply(celsius)) / 
                                             (243.04 + weather.DewPoint.apply(celsius))) / 
                                      np.exp((17.625 * weather.Tavg.apply(celsius)) / 
                                             (243.04 + weather.Tavg.apply(celsius))))

## Convert DewPoint to Celsius

In [11]:
weather['DewPointC'] = weather['DewPoint'].apply(celsius)

## Add rain boolean

- TS THUNDERSTORM
- GR HAIL
- RA RAIN
- DZ DRIZZLE
- SH SHOWER

In [12]:
def filter_codesum(x):
    for code in rain:
        if code in x:
            return 1
        else:
            return 0

In [13]:
rain = ['RA', 'DZ', 'SH', 'TS', 'GR']

In [14]:
weather['rain'] = weather.CodeSum.apply(filter_codesum)

## Add Ideal Environment boolean

In [15]:
weather.loc[(weather['TavgC'] >= 10) & (weather['TavgC'] <= 35) & 
            (weather['relative_humidity'] <= 95) & (weather['relative_humidity'] >= 40) & 
            (weather['AvgSpeed'] <= 6), 
            'ideal_environment']=1

weather.ideal_environment.fillna(0, inplace=True)

## Add Ideal Feeding Conditions boolean

In [16]:
weather.loc[(weather['TavgC'] >= 15) & (weather['TavgC'] <= 30) & 
            (weather['relative_humidity'] >= 8) & (weather['relative_humidity'] <= 72) & 
            (weather['AvgSpeed'] <= 6), 
            'ideal_feeding']=1

weather.ideal_feeding.fillna(0, inplace=True)

## Average values to drop station

In [17]:
new_weather_df = weather.groupby('Date')['TavgC','TmaxC','TminC','DewPointC','relative_humidity',
                                         'rain','ideal_environment','ideal_feeding'].mean()

## Calculate Photoperiod

In [18]:
sun = weather[weather.Station == 1][['Date','Sunset','Sunrise']]

In [19]:
sun['sundiff'] = sun.apply(lambda x: int(x.Sunset) - int(x.Sunrise), axis=1)

In [20]:
sun['photoperiod'] =sun.sundiff.map(lambda x: int(str(x)[:2]) + int(str(x)[-2:])/60)

In [21]:
sun.index = sun.Date

In [22]:
sun = sun.drop(['Date','Sunset','Sunrise','sundiff'], axis=1)

## Join Photoperiod with new_weather_df

In [23]:
new_weather_df['Date'] = new_weather_df.index

In [24]:
new_weather_df = new_weather_df.join(sun, on='Date')

/Users/yibingchen/anaconda3/envs/tensorflow/lib/python3.6/site-packages/pandas/core/frame.py:6326: FutureWarning: 'Date' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  rsuffix=rsuffix, sort=sort)


In [25]:
new_weather_df = new_weather_df.drop(['Date'], axis=1)

In [26]:
new_weather_df.head()

,TavgC,TmaxC,TminC,DewPointC,relative_humidity,rain,ideal_environment,ideal_feeding,photoperiod
Date,,,,,,,,,
2007-05-01,19.722222,28.611111,10.555556,10.555556,55.482501,0.0,0.0,0.0,14.016667
2007-05-02,10.833333,15.277778,5.833333,5.555556,69.905385,0.0,0.0,0.0,14.050000
2007-05-03,13.888889,19.166667,8.333333,4.444444,52.966288,0.0,0.0,0.0,14.083333
2007-05-04,16.111111,22.222222,10.000000,5.277778,48.837189,0.5,0.0,0.0,14.133333
2007-05-05,15.555556,18.888889,11.944444,3.611111,44.842896,0.0,0.0,0.0,14.166667


## Create Weather Features for Last 14 Days

In [27]:
def time_machine(df, column, days, bc = 0):
    new_df = df[[column]]
    for i in range(1,days+1):
        new_col_name = column + "_" + str(i) +"_back"
        new_col = one_day_back(df, column, i, bc)
        new_df[new_col_name] = new_col
    new_df.drop(column, 1, inplace=True)
    df = df.join(new_df)
    return df

def one_day_back(df, column, day, bc):
    col = df[column].tolist()
    for i in range(day):
        del col[-1]
        col.insert(0, bc)
    return col

In [28]:
new_weather_df.columns

Index(['TavgC', 'TmaxC', 'TminC', 'DewPointC', 'relative_humidity', 'rain',
       'ideal_environment', 'ideal_feeding', 'photoperiod'],
      dtype='object')

In [29]:
with open('../Cleansed_Data/weather_df_comp.pkl', 'wb') as f:
    pickle.dump(new_weather_df, f)

In [30]:
time_machine(new_weather_df, 'DewPointC', 14)

/Users/yibingchen/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,TavgC,TmaxC,TminC,DewPointC,relative_humidity,rain,ideal_environment,ideal_feeding,photoperiod,DewPointC_1_back,...,DewPointC_5_back,DewPointC_6_back,DewPointC_7_back,DewPointC_8_back,DewPointC_9_back,DewPointC_10_back,DewPointC_11_back,DewPointC_12_back,DewPointC_13_back,DewPointC_14_back
Date,,,,,,,,,,,,,,,,,,,,,
2007-05-01,19.722222,28.611111,10.555556,10.555556,55.482501,0.0,0.0,0.0,14.016667,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2007-05-02,10.833333,15.277778,5.833333,5.555556,69.905385,0.0,0.0,0.0,14.050000,10.555556,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2007-05-03,13.888889,19.166667,8.333333,4.444444,52.966288,0.0,0.0,0.0,14.083333,5.555556,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2007-05-04,16.111111,22.222222,10.000000,5.277778,48.837189,0.5,0.0,0.0,14.133333,4.444444,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2007-05-05,15.555556,18.888889,11.944444,3.611111,44.842896,0.0,0.0,0.0,14.166667,5.277778,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2007-05-06,15.277778,20.000000,10.277778,-1.111111,32.524713,0.0,0.0,0.0,14.216667,3.611111,...,10.555556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2007-05-07,18.888889,28.611111,9.166667,4.444444,38.604051,0.5,0.0,0.0,14.250000,-1.111111,...,5.555556,10.555556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2007-05-08,20.555556,27.222222,13.888889,14.166667,66.882020,0.0,1.0,1.0,14.300000,4.444444,...,4.444444,5.555556,10.555556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2007-05-09,20.833333,24.722222,16.666667,15.277778,70.530226,0.0,0.5,0.5,14.333333,14.166667,...,5.277778,4.444444,5.555556,10.555556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [31]:
for col in new_weather_df.columns:
    new_weather_df = time_machine(new_weather_df, col, 14)


/Users/yibingchen/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/yibingchen/anaconda3/envs/tensorflow/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


## Export new_weather_df

In [32]:
with open('../Cleansed_Data/new_weather_df.pkl', 'wb') as f:
    pickle.dump(new_weather_df, f)